# Infer-5-Skills-IRT : Evaluation de Competences et Psychometrie

**Serie** : Programmation Probabiliste avec Infer.NET (5/13)  
**Duree estimee** : 60 minutes  
**Prerequis** : Infer-4-Bayesian-Networks

---

## Objectifs

- Comprendre les modeles d'evaluation cognitive
- Implementer le modele IRT (Item Response Theory) Difficulty-Ability
- Construire le modele DINA (Noisy-And) pour competences multiples
- Gerer les relations many-to-many entre competences et questions
- Estimer les parametres slip et guess

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-4-Bayesian-Networks](Infer-4-Bayesian-Networks.ipynb) | [Infer-6-TrueSkill](Infer-6-TrueSkill.ipynb) |

---

## 1. Configuration

Nous preparons l'environnement pour les modeles d'evaluation de competences. Ces modeles psychometriques (IRT, DINA) utilisent des variables latentes pour representer les capacites non observees des etudiants et les difficultes des questions.

In [1]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


## 2. Introduction a l'Evaluation Cognitive

### Probleme

Comment evaluer les competences d'un etudiant a partir de ses reponses a un test ?

### Defis

| Defi | Description |
|------|-------------|
| **Capacite latente** | La competence n'est pas directement observable |
| **Bruit** | Les reponses peuvent etre correctes par chance ou incorrectes par erreur |
| **Difficulte variable** | Les questions ont des difficultes differentes |
| **Competences multiples** | Une question peut necessiter plusieurs competences |

### Approches

| Modele | Caracteristique |
|--------|----------------|
| **IRT classique** | Capacite unidimensionnelle + difficulte des questions |
| **DINA** | Competences discretes multiples + matrice Q |
| **Bayesien hierarchique** | Parametres appris de maniere adaptative |

## 3. Modele IRT : Difficulty-Ability

### Formulation

$$P(\text{correct}_{ij}) = \sigma(\text{capacite}_i - \text{difficulte}_j)$$

Ou $\sigma$ est la fonction logistique (ou probit dans notre cas).

### Structure

```
capacite[i] ~ N(0, 1)    pour chaque etudiant
difficulte[j] ~ N(0, 1)  pour chaque question

avantage[i,j] = capacite[i] - difficulte[j]
reponse[i,j] ~ Probit(avantage[i,j])
```

In [2]:
// Modele IRT Difficulty-Ability

int nEtudiants = 10;
int nQuestions = 5;

// Donnees simulees : matrice de reponses (true = correct)
bool[,] reponses = new bool[,] {
    // Q1    Q2     Q3     Q4     Q5
    { true,  true,  true,  false, false },  // Etudiant 1 (bon)
    { true,  true,  false, false, false },  // Etudiant 2
    { true,  false, false, false, false },  // Etudiant 3 (faible)
    { true,  true,  true,  true,  false },  // Etudiant 4 (tres bon)
    { false, false, false, false, false },  // Etudiant 5 (tres faible)
    { true,  true,  false, true,  false },  // Etudiant 6
    { true,  true,  true,  false, true },   // Etudiant 7
    { true,  false, true,  false, false },  // Etudiant 8
    { true,  true,  true,  true,  true },   // Etudiant 9 (excellent)
    { false, true,  false, false, false }   // Etudiant 10
};

// Definition du modele
Range etudiant = new Range(nEtudiants).Named("etudiant");
Range question = new Range(nQuestions).Named("question");

// Capacites latentes des etudiants
VariableArray<double> capacite = Variable.Array<double>(etudiant).Named("capacite");
capacite[etudiant] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(etudiant);

// Difficultes des questions
VariableArray<double> difficulte = Variable.Array<double>(question).Named("difficulte");
difficulte[question] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(question);

// Discrimination (bruit)
Variable<double> discrimination = Variable.GammaFromShapeAndScale(2, 0.5).Named("discrimination");

// Reponses observees
VariableArray2D<bool> reponseVar = Variable.Array<bool>(etudiant, question).Named("reponse");

using (Variable.ForEach(etudiant))
{
    using (Variable.ForEach(question))
    {
        Variable<double> avantage = capacite[etudiant] - difficulte[question];
        Variable<double> avantageBruite = Variable.GaussianFromMeanAndPrecision(avantage, discrimination);
        reponseVar[etudiant, question] = (avantageBruite > 0);
    }
}

// Observations
reponseVar.ObservedValue = reponses;

Console.WriteLine("Modele IRT defini.");

Modele IRT defini.


In [3]:
// Inference
InferenceEngine moteurIRT = new InferenceEngine(new ExpectationPropagation());
moteurIRT.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Gaussian[] capacitePost = moteurIRT.Infer<Gaussian[]>(capacite);
Gaussian[] difficultePost = moteurIRT.Infer<Gaussian[]>(difficulte);

Console.WriteLine("=== Capacites des etudiants (IRT) ===");
for (int i = 0; i < nEtudiants; i++)
{
    int nCorrect = 0;
    for (int j = 0; j < nQuestions; j++) if (reponses[i, j]) nCorrect++;
    Console.WriteLine($"Etudiant {i+1} : capacite = {capacitePost[i].GetMean():F2} +/- {Math.Sqrt(capacitePost[i].GetVariance()):F2} (score: {nCorrect}/{nQuestions})");
}

Console.WriteLine("\n=== Difficultes des questions ===");
for (int j = 0; j < nQuestions; j++)
{
    int nReussi = 0;
    for (int i = 0; i < nEtudiants; i++) if (reponses[i, j]) nReussi++;
    Console.WriteLine($"Question {j+1} : difficulte = {difficultePost[j].GetMean():F2} (taux reussite: {nReussi}/{nEtudiants})");
}

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
=== Capacites des etudiants (IRT) ===
Etudiant 1 : capacite = 0,26 +/- 0,60 (score: 3/5)
Etudiant 2 : capacite = -0,26 +/- 0,60 (score: 2/5)
Etudiant 3 : capacite = -0,81 +/- 0,62 (score: 1/5)
Etudiant 4 : capacite = 0,81 +/- 0,62 (score: 4/5)
Etudiant 5 : capacite = -1,44 +/- 0,67 (score: 0/5)
Etudiant 6 : capacite = 0,27 +/- 0,59 (score: 3/5)
Etudiant 7 : capacite = 0,81 +/- 0,61 (score: 4/5)
Etudiant 8 : capacite = -0,27 +/- 0,59 (score: 2/5)
Etudiant 9 : capacite = 1,44 +/- 0,67 (score: 5/5)
Etudiant 10 : capacite = -0,81 +/- 0,61 (score: 1/5)

=== Difficultes des questions ===
Question 1 : difficulte = -1,00 (taux reussite: 8/10)
Question 2 : difficulte = -0,64 (taux reussite: 7/10)
Question 3 : difficulte = 0,00 (taux reussite: 5/10)
Question 4 : difficulte = 0,64 (taux reussite: 3/10)
Question 5 : difficulte = 1,00 (taux reussite: 2/10)


### Analyse détaillée des résultats IRT

**Observation clé** : La capacité estimée est **fortement corrélée** au score brut, mais pas parfaitement identique.

| Étudiant | Score | Capacité | Observation |
|----------|-------|----------|-------------|
| E5 | 0/5 | -1.44 | Le plus faible |
| E9 | 5/5 | +1.44 | Le plus fort |
| E1, E6 | 3/5 | ~+0.27 | Niveau moyen |

**Ordre des questions par difficulté** : Q1 < Q2 < Q3 < Q4 < Q5

Cela correspond aux taux de réussite observés (80%, 70%, 50%, 30%, 20%).

**Avantage du modèle IRT sur le score brut** :
- L'**écart-type** reflète l'incertitude (plus grande pour les scores intermédiaires)
- La capacité tient compte de la **difficulté des questions réussies**
- Un étudiant qui réussit Q5 (difficile) a une capacité plus élevée qu'un autre avec le même score mais sur des questions faciles

## 3bis. Évaluation du Modèle : Courbes ROC

Une **courbe ROC** (Receiver Operating Characteristic) permet d'évaluer la qualité des prédictions d'un modèle de classification. Pour l'IRT, nous pouvons prédire si un étudiant va répondre correctement à une question et comparer aux réponses réelles.

### Métriques d'évaluation

| Métrique | Formule | Interprétation |
|----------|---------|----------------|
| **AUC** | Aire sous la courbe ROC | 0.5 = aléatoire, 1.0 = parfait |
| **TPR** | TP / (TP + FN) | Taux de vrais positifs (sensibilité) |
| **FPR** | FP / (FP + TN) | Taux de faux positifs (1 - spécificité) |

In [ ]:
// Calcul de la courbe ROC pour le modele IRT

Console.WriteLine("=== Evaluation ROC du modele IRT ===\n");

// Calculer les probabilites de reponse correcte pour chaque paire (etudiant, question)
List<(double prob, bool actual)> predictions = new List<(double, bool)>();

for (int i = 0; i < nEtudiants; i++)
{
    for (int j = 0; j < nQuestions; j++)
    {
        // P(correct) approxime par la fonction probit
        double avantage = capacitePost[i].GetMean() - difficultePost[j].GetMean();
        // Approximation de Phi (CDF normale) par fonction logistique
        double probCorrect = 1.0 / (1.0 + Math.Exp(-1.7 * avantage));
        predictions.Add((probCorrect, reponses[i, j]));
    }
}

// Trier par probabilite decroissante
var sorted = predictions.OrderByDescending(p => p.prob).ToList();

// Calculer les points de la courbe ROC
int totalPos = sorted.Count(p => p.actual);
int totalNeg = sorted.Count - totalPos;

List<(double fpr, double tpr)> rocPoints = new List<(double, double)>();
rocPoints.Add((0.0, 0.0));

int tp = 0, fp = 0;
double lastProb = 1.0;

foreach (var pred in sorted)
{
    if (pred.actual) tp++;
    else fp++;
    
    double tpr = (double)tp / totalPos;
    double fpr = (double)fp / totalNeg;
    rocPoints.Add((fpr, tpr));
}

// Calcul de l'AUC (methode des trapezes)
double auc = 0;
for (int i = 1; i < rocPoints.Count; i++)
{
    double width = rocPoints[i].fpr - rocPoints[i-1].fpr;
    double height = (rocPoints[i].tpr + rocPoints[i-1].tpr) / 2;
    auc += width * height;
}

Console.WriteLine($"AUC (Area Under Curve) : {auc:F3}");
Console.WriteLine($"  0.5 = aleatoire, 1.0 = parfait");
Console.WriteLine($"  Interpretation : {(auc > 0.9 ? "Excellent" : auc > 0.8 ? "Bon" : auc > 0.7 ? "Acceptable" : "Faible")}");

// Afficher quelques points de la courbe
Console.WriteLine("\nPoints cles de la courbe ROC :");
Console.WriteLine("| Seuil | FPR  | TPR  |");
Console.WriteLine("|-------|------|------|");
double[] seuils = { 0.9, 0.7, 0.5, 0.3, 0.1 };
foreach (double seuil in seuils)
{
    int tpS = predictions.Count(p => p.prob >= seuil && p.actual);
    int fpS = predictions.Count(p => p.prob >= seuil && !p.actual);
    double tprS = (double)tpS / totalPos;
    double fprS = (double)fpS / totalNeg;
    Console.WriteLine($"| {seuil:F1}   | {fprS:F2} | {tprS:F2} |");
}

### Interprétation de la Courbe ROC

**AUC (Area Under Curve)** mesure la capacité discriminante du modèle :

| AUC | Interprétation | Action suggérée |
|-----|----------------|-----------------|
| 0.9-1.0 | Excellent | Le modèle est très fiable |
| 0.8-0.9 | Bon | Utilisable en production |
| 0.7-0.8 | Acceptable | À améliorer si possible |
| 0.5-0.7 | Faible | Revoir le modèle |

**Lecture de la table** :
- **Seuil 0.7** : Si on prédit "correct" quand P(correct) > 0.7
  - TPR = % de vraies bonnes réponses capturées
  - FPR = % de fausses alertes (mauvaises réponses prédites correctes)

**Application pour l'IRT** :

Le modèle IRT devrait avoir une AUC élevée car :
1. Il capture la **vraie capacité** de chaque étudiant
2. Il tient compte de la **difficulté** de chaque question
3. La prédiction P(correct) = f(capacité - difficulté) est bien calibrée

> **Note** : Avec seulement 50 observations (10 étudiants × 5 questions), l'AUC peut être bruitée. Un test plus grand donnerait une estimation plus stable.

## 4. Modele DINA : Competences Discretes

### Motivation

Le modele IRT suppose une capacite **unidimensionnelle**. En realite, un test peut evaluer **plusieurs competences**.

### Modele DINA (Deterministic Input, Noisy And)

- Chaque etudiant possede ou non chaque competence (variable binaire)
- Chaque question necessite un sous-ensemble de competences (matrice Q)
- Reponse correcte si **toutes** les competences requises sont presentes
- Avec des erreurs slip et guess

### Parametres

| Parametre | Description |
|-----------|-------------|
| **Slip** | P(incorrect \| a toutes les competences) - erreur d'inattention |
| **Guess** | P(correct \| manque une competence) - reponse au hasard |

In [4]:
// Modele DINA simplifie

int nEtud = 8;
int nQuest = 6;
int nCompetences = 3;

// Matrice Q : quelles competences sont requises pour chaque question
// Q1 : C1 seulement
// Q2 : C2 seulement
// Q3 : C3 seulement
// Q4 : C1 et C2
// Q5 : C2 et C3
// Q6 : C1, C2 et C3
bool[,] matriceQ = new bool[,] {
    // C1    C2     C3
    { true,  false, false },  // Q1
    { false, true,  false },  // Q2
    { false, false, true  },  // Q3
    { true,  true,  false },  // Q4
    { false, true,  true  },  // Q5
    { true,  true,  true  }   // Q6
};

// Donnees : reponses des etudiants
bool[,] repDINA = new bool[,] {
    // Q1    Q2     Q3     Q4     Q5     Q6
    { true,  true,  true,  true,  true,  true  },  // E1 : a tout
    { true,  true,  false, true,  false, false },  // E2 : C1, C2 seulement
    { true,  false, true,  false, false, false },  // E3 : C1, C3 seulement
    { false, true,  true,  false, true,  false },  // E4 : C2, C3 seulement
    { true,  false, false, false, false, false },  // E5 : C1 seulement
    { false, true,  false, false, false, false },  // E6 : C2 seulement
    { false, false, true,  false, false, false },  // E7 : C3 seulement
    { false, false, false, false, false, false }   // E8 : rien
};

Console.WriteLine("Donnees DINA definies.");
Console.WriteLine("\nMatrice Q (competences requises par question) :");
for (int q = 0; q < nQuest; q++)
{
    string req = "";
    for (int c = 0; c < nCompetences; c++)
        if (matriceQ[q, c]) req += $"C{c+1} ";
    Console.WriteLine($"  Q{q+1} : {req}");
}

Donnees DINA definies.

Matrice Q (competences requises par question) :
  Q1 : C1 
  Q2 : C2 
  Q3 : C3 
  Q4 : C1 C2 
  Q5 : C2 C3 
  Q6 : C1 C2 C3 


In [5]:
// Note sur l'implementation DINA complete
// Le modele DINA avec AND dynamique sur les competences requises est complexe a
// implementer dans Infer.NET en raison des limitations sur SetTo dans les blocs conditionnels.
// Nous utilisons ci-dessous une version simplifiee qui illustre les concepts.

// Affichage de la structure du modele DINA
Console.WriteLine("=== Structure du modele DINA ===");
Console.WriteLine();
Console.WriteLine("Pour chaque etudiant e et question q :");
Console.WriteLine("  1. Verifier si e a toutes les competences requises par q (selon matrice Q)");
Console.WriteLine("  2. Si oui : P(correct) = 1 - slip");
Console.WriteLine("  3. Si non : P(correct) = guess");
Console.WriteLine();
Console.WriteLine("Parametres a estimer :");
Console.WriteLine("  - Competences[e,c] : chaque etudiant a-t-il chaque competence ?");
Console.WriteLine("  - slip : probabilite d'erreur malgre les competences");
Console.WriteLine("  - guess : probabilite de reussite sans les competences");

=== Structure du modele DINA ===

Pour chaque etudiant e et question q :
  1. Verifier si e a toutes les competences requises par q (selon matrice Q)
  2. Si oui : P(correct) = 1 - slip
  3. Si non : P(correct) = guess

Parametres a estimer :
  - Competences[e,c] : chaque etudiant a-t-il chaque competence ?
  - slip : probabilite d'erreur malgre les competences
  - guess : probabilite de reussite sans les competences


> **Note** : Le modele DINA complet avec Variable.ForEach et AND dynamique est complexe dans Infer.NET. Nous allons utiliser une version simplifiee ci-dessous.

In [6]:
// Version simplifiee : modele DINA pour une question specifique

// Question Q4 necessite C1 ET C2
Variable<bool> aC1 = Variable.Bernoulli(0.5).Named("aC1");
Variable<bool> aC2 = Variable.Bernoulli(0.5).Named("aC2");

Variable<double> slipQ4 = Variable.Beta(1, 9);
Variable<double> guessQ4 = Variable.Beta(1, 9);

Variable<bool> toutesCompQ4 = aC1 & aC2;
Variable<bool> reponseQ4 = Variable.New<bool>();

using (Variable.If(toutesCompQ4))
{
    reponseQ4.SetTo(!Variable.Bernoulli(slipQ4));
}
using (Variable.IfNot(toutesCompQ4))
{
    reponseQ4.SetTo(Variable.Bernoulli(guessQ4));
}

// Observation : l'etudiant a repondu correctement
reponseQ4.ObservedValue = true;

InferenceEngine mDINA = new InferenceEngine();
mDINA.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== DINA : Inference des competences ===");
Console.WriteLine($"Question Q4 necessite C1 ET C2");
Console.WriteLine($"Observation : reponse correcte\n");
Console.WriteLine($"P(C1) = {mDINA.Infer<Bernoulli>(aC1).GetProbTrue():F3}");
Console.WriteLine($"P(C2) = {mDINA.Infer<Bernoulli>(aC2).GetProbTrue():F3}");
Console.WriteLine($"P(C1 ET C2) = {mDINA.Infer<Bernoulli>(toutesCompQ4).GetProbTrue():F3}");
Console.WriteLine($"\nSlip estime : {mDINA.Infer<Beta>(slipQ4).GetMean():F3}");
Console.WriteLine($"Guess estime : {mDINA.Infer<Beta>(guessQ4).GetMean():F3}");

=== DINA : Inference des competences ===
Question Q4 necessite C1 ET C2
Observation : reponse correcte

Compiling model...done.
P(C1) = 0,833
P(C2) = 0,833
P(C1 ET C2) = 0,750

Slip estime : 0,093
Guess estime : 0,120


### Analyse du modèle DINA simplifié

**Résultats** :
- P(C1) = P(C2) = **0.833** (augmenté depuis le prior de 0.5)
- P(C1 ET C2) = **0.750**
- Slip estimé : ~0.09, Guess estimé : ~0.12

**Interprétation** :

L'observation d'une réponse correcte à Q4 (qui nécessite C1 ET C2) favorise les deux compétences :

1. **Sans guess** : La seule façon de répondre correctement est d'avoir C1 ET C2
2. **Avec guess** : Il y a une petite probabilité (~10%) de réussir sans les compétences

Le modèle infère que l'étudiant a **probablement** les deux compétences, mais conserve une incertitude résiduelle due au guess possible.

> **Remarque** : P(C1 ET C2) = 0.75 ≠ P(C1) × P(C2) = 0.69 car les observations créent une **dépendance** entre C1 et C2 (explaining away : si l'un est absent, l'autre doit être présent pour expliquer la réussite par guess).

## 5. Relations Many-to-Many

### Probleme

Dans le modele DINA, une question peut necessiter **plusieurs competences**, et une competence peut etre evaluee par **plusieurs questions**.

### Representation avec Subarray

Infer.NET permet d'utiliser `Variable.Subarray` pour extraire les competences requises pour chaque question.

In [7]:
// Demonstration avec plusieurs etudiants et questions

int nE = 4;
int nC = 3;

// Prior : chaque etudiant a 50% de chance d'avoir chaque competence
Range rE = new Range(nE);
Range rC = new Range(nC);

VariableArray2D<bool> competences = Variable.Array<bool>(rE, rC);
competences[rE, rC] = Variable.Bernoulli(0.5).ForEach(rE, rC);

// Simuler des observations sur plusieurs questions
// Q1 necessite C1 seulement
// Q2 necessite C1 ET C2

// Observations pour Q1 (C1 seulement)
bool[] obsQ1 = { true, true, false, true };  // E1, E2, E4 ont repondu correctement

for (int e = 0; e < nE; e++)
{
    Variable<bool> toutQ1 = competences[e, 0];  // Seulement C1
    Variable<bool> repQ1 = Variable.New<bool>();
    using (Variable.If(toutQ1))
    {
        repQ1.SetTo(Variable.Bernoulli(0.9));  // 1 - slip
    }
    using (Variable.IfNot(toutQ1))
    {
        repQ1.SetTo(Variable.Bernoulli(0.1));  // guess
    }
    repQ1.ObservedValue = obsQ1[e];
}

// Observations pour Q2 (C1 ET C2)
bool[] obsQ2 = { true, false, false, true };

for (int e = 0; e < nE; e++)
{
    Variable<bool> toutQ2 = competences[e, 0] & competences[e, 1];
    Variable<bool> repQ2 = Variable.New<bool>();
    using (Variable.If(toutQ2))
    {
        repQ2.SetTo(Variable.Bernoulli(0.9));
    }
    using (Variable.IfNot(toutQ2))
    {
        repQ2.SetTo(Variable.Bernoulli(0.1));
    }
    repQ2.ObservedValue = obsQ2[e];
}

InferenceEngine mMany = new InferenceEngine();
mMany.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Bernoulli[,] compPost = mMany.Infer<Bernoulli[,]>(competences);

Console.WriteLine("=== Inference des competences (many-to-many) ===");
Console.WriteLine("Q1 necessite C1, Q2 necessite C1 ET C2\n");
Console.WriteLine("Observations : Q1=[T,T,F,T], Q2=[T,F,F,T]\n");

for (int e = 0; e < nE; e++)
{
    Console.Write($"Etudiant {e+1} : ");
    for (int c = 0; c < nC; c++)
    {
        Console.Write($"C{c+1}={compPost[e,c].GetProbTrue():F2} ");
    }
    Console.WriteLine();
}

Compiling model...done.
=== Inference des competences (many-to-many) ===
Q1 necessite C1, Q2 necessite C1 ET C2

Observations : Q1=[T,T,F,T], Q2=[T,F,F,T]

Etudiant 1 : C1=0,98 C2=0,89 C3=0,50 
Etudiant 2 : C1=0,83 C2=0,17 C3=0,50 
Etudiant 3 : C1=0,06 C2=0,48 C3=0,50 
Etudiant 4 : C1=0,98 C2=0,89 C3=0,50 


### Analyse des inférences many-to-many

**Pattern des résultats** :

| Étudiant | Q1(C1) | Q2(C1∧C2) | C1 inféré | C2 inféré | Interprétation |
|----------|--------|-----------|-----------|-----------|----------------|
| E1 | ✓ | ✓ | **0.98** | **0.89** | A probablement les deux |
| E2 | ✓ | ✗ | 0.83 | **0.17** | A C1, pas C2 |
| E3 | ✗ | ✗ | **0.06** | 0.48 | N'a pas C1 |
| E4 | ✓ | ✓ | **0.98** | **0.89** | A probablement les deux |

**Points clés** :

1. **C3 reste à 0.50** pour tous les étudiants car aucune question ne l'évalue
2. **E2** : Réussit Q1 (C1 seul) mais échoue Q2 (C1∧C2) → P(C2) chute à 0.17
3. **E1 vs E4** : Mêmes observations, mêmes inférences (cohérence du modèle)

> **Puissance du modèle** : Les questions multi-compétences (Q2) permettent de **discriminer** entre les compétences mieux qu'avec des questions à compétence unique.

## 6. Estimation des Parametres Slip/Guess

### Objectif

Apprendre les parametres slip et guess a partir des donnees.

### Approche

- Utiliser des priors Beta sur slip et guess
- L'inference met a jour ces distributions

In [8]:
// Estimation de slip/guess avec donnees multiples

// Simuler des donnees ou on connait les vraies competences
int nObs = 20;
Random rng = new Random(42);

double vraiSlip = 0.1;
double vraiGuess = 0.2;

// Generer des donnees
bool[] vraiComp = new bool[nObs];    // Vrai etat de competence
bool[] obsRep = new bool[nObs];       // Reponse observee

for (int i = 0; i < nObs; i++)
{
    vraiComp[i] = rng.NextDouble() < 0.6;  // 60% ont la competence
    if (vraiComp[i])
    {
        obsRep[i] = rng.NextDouble() > vraiSlip;  // Correct sauf slip
    }
    else
    {
        obsRep[i] = rng.NextDouble() < vraiGuess;  // Incorrect sauf guess
    }
}

// Modele pour estimer slip et guess
Variable<double> slipEst = Variable.Beta(1, 1).Named("slip");
Variable<double> guessEst = Variable.Beta(1, 1).Named("guess");
Variable<double> pComp = Variable.Beta(1, 1).Named("pComp");

Range rObs = new Range(nObs);
VariableArray<bool> comp = Variable.Array<bool>(rObs);
VariableArray<bool> rep = Variable.Array<bool>(rObs);

comp[rObs] = Variable.Bernoulli(pComp).ForEach(rObs);

using (Variable.ForEach(rObs))
{
    using (Variable.If(comp[rObs]))
    {
        rep[rObs] = !Variable.Bernoulli(slipEst);
    }
    using (Variable.IfNot(comp[rObs]))
    {
        rep[rObs] = Variable.Bernoulli(guessEst);
    }
}

rep.ObservedValue = obsRep;

InferenceEngine mSlipGuess = new InferenceEngine(new ExpectationPropagation());
mSlipGuess.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Estimation Slip/Guess ===");
Console.WriteLine($"Vraies valeurs : slip={vraiSlip}, guess={vraiGuess}\n");
Console.WriteLine($"Estimations :");
Console.WriteLine($"  Slip : {mSlipGuess.Infer<Beta>(slipEst)}");
Console.WriteLine($"  Guess : {mSlipGuess.Infer<Beta>(guessEst)}");
Console.WriteLine($"  P(competence) : {mSlipGuess.Infer<Beta>(pComp)}");

=== Estimation Slip/Guess ===
Vraies valeurs : slip=0,1, guess=0,2

Estimations :
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
  Slip : Beta(3,871,7,698)[mean=0,3346]
  Guess : Beta(7,698,3,871)[mean=0,6654]
  P(competence) : Beta(1,1)[mean=0,5]


## 7. Comparaison IRT vs DINA

| Aspect | IRT | DINA |
|--------|-----|------|
| **Capacite** | Continue, unidimensionnelle | Discrete, multidimensionnelle |
| **Interpretation** | "Niveau global" | "Competences specifiques" |
| **Complexite** | Simple, robuste | Plus complexe, informatif |
| **Utilisation** | Tests standardises | Diagnostic pedagogique |
| **Inference** | Plus facile | Necessite matrice Q |

## 8. Exercice : Evaluer un Nouvel Etudiant

### Enonce

Un nouvel etudiant passe un test de 5 questions. Le test evalue 2 competences :
- Q1, Q2 : Competence 1 seulement
- Q3, Q4 : Competence 2 seulement
- Q5 : Competences 1 ET 2

Resultats : Q1=correct, Q2=correct, Q3=incorrect, Q4=correct, Q5=incorrect

**Question** : Quelles sont les probabilites que l'etudiant possede C1 et C2 ?

In [9]:
// EXERCICE : Evaluation d'un nouvel etudiant

Variable<bool> c1 = Variable.Bernoulli(0.5).Named("C1");
Variable<bool> c2 = Variable.Bernoulli(0.5).Named("C2");

double slip = 0.1;
double guess = 0.15;

// Q1 : C1 seulement -> correct
Variable<bool> rQ1 = Variable.New<bool>();
using (Variable.If(c1)) { rQ1.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c1)) { rQ1.SetTo(Variable.Bernoulli(guess)); }
rQ1.ObservedValue = true;

// Q2 : C1 seulement -> correct
Variable<bool> rQ2 = Variable.New<bool>();
using (Variable.If(c1)) { rQ2.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c1)) { rQ2.SetTo(Variable.Bernoulli(guess)); }
rQ2.ObservedValue = true;

// Q3 : C2 seulement -> incorrect
Variable<bool> rQ3 = Variable.New<bool>();
using (Variable.If(c2)) { rQ3.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c2)) { rQ3.SetTo(Variable.Bernoulli(guess)); }
rQ3.ObservedValue = false;

// Q4 : C2 seulement -> correct
Variable<bool> rQ4 = Variable.New<bool>();
using (Variable.If(c2)) { rQ4.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c2)) { rQ4.SetTo(Variable.Bernoulli(guess)); }
rQ4.ObservedValue = true;

// Q5 : C1 ET C2 -> incorrect
Variable<bool> c1etc2 = c1 & c2;
Variable<bool> rQ5 = Variable.New<bool>();
using (Variable.If(c1etc2)) { rQ5.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c1etc2)) { rQ5.SetTo(Variable.Bernoulli(guess)); }
rQ5.ObservedValue = false;

InferenceEngine mEx = new InferenceEngine();
mEx.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Evaluation du nouvel etudiant ===");
Console.WriteLine("Resultats : Q1=T, Q2=T, Q3=F, Q4=T, Q5=F\n");
Console.WriteLine($"P(C1) = {mEx.Infer<Bernoulli>(c1).GetProbTrue():F3}");
Console.WriteLine($"P(C2) = {mEx.Infer<Bernoulli>(c2).GetProbTrue():F3}");
Console.WriteLine($"\nInterpretation :");
Console.WriteLine("- L'etudiant a probablement C1 (2/2 correct sur questions C1)");
Console.WriteLine("- C2 est moins certain (1/2 correct, et Q5 echoue)");

=== Evaluation du nouvel etudiant ===
Resultats : Q1=T, Q2=T, Q3=F, Q4=T, Q5=F

Compiling model...done.
P(C1) = 0,958
P(C2) = 0,091

Interpretation :
- L'etudiant a probablement C1 (2/2 correct sur questions C1)
- C2 est moins certain (1/2 correct, et Q5 echoue)


### Analyse du diagnostic de l'étudiant

**Résultats** :
- P(C1) = **0.958** → L'étudiant a très probablement C1
- P(C2) = **0.091** → L'étudiant n'a probablement PAS C2

**Raisonnement du modèle** :

| Observation | Impact sur C1 | Impact sur C2 |
|-------------|---------------|---------------|
| Q1=T (C1) | ↑ Forte hausse | - |
| Q2=T (C1) | ↑ Confirmation | - |
| Q3=F (C2) | - | ↓ Baisse |
| Q4=T (C2) | - | ↑ Légère hausse |
| Q5=F (C1∧C2) | ↓ Légère baisse | ↓ Forte baisse |

**Clé** : L'échec à Q5 (C1∧C2) avec P(C1) élevé implique que C2 est probablement manquant (explaining away).

> **Application pédagogique** : Ce diagnostic permet de cibler la remédiation sur C2 spécifiquement, plutôt que de faire reprendre tout le cours à l'étudiant.

## 9. Resume

| Concept | Description |
|---------|-------------|
| **IRT** | Modele capacite-difficulte pour tests unidimensionnels |
| **DINA** | Modele a competences discretes multiples |
| **Matrice Q** | Definition des competences requises par question |
| **Slip** | Erreur d'inattention (correct -> incorrect) |
| **Guess** | Reponse au hasard (incorrect -> correct) |
| **Many-to-many** | Questions multiples, competences multiples |

---

## Prochaine etape

Dans [Infer-6-TrueSkill](Infer-6-TrueSkill.ipynb), nous explorerons :

- Le systeme de classement TrueSkill (Xbox Live)
- L'apprentissage en ligne des competences
- La gestion des matchs nuls
- La visualisation du message passing